# Data science for business project
This project is made by 
- Drago Emanuele
- Lambrughi Achille

The aim of this project is to retrive, clean, analyze and develop a machine learning model starting from some given dataset.

The dataset considered are available at the following link:
- https://www.dati.lombardia.it/Attivit-Produttive/Rapporti-di-lavoro-attivati/qbau-cyuc


In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl, matplotlib.pyplot as plt
from pathlib import PurePath
from datetime import datetime, timedelta

In [ ]:
rap_lavoro_attivati = pd.read_csv(PurePath('dataset', 'Rapporti_di_lavoro_attivati.csv'),parse_dates=['DATA'])
print([rap_lavoro_attivati.info()])

In [ ]:
def series_to_set(column, source_df):
    SET = set()
    for elem in source_df[column]:
        SET.add(elem)
    return SET

test_dataset = series_to_set('MODALITALAVORO', rap_lavoro_attivati)
test_dataset

# Dataset analysis
In this section will be analyzed the content of the datasets. The first group of datasets contain work relations started\ended\changed\extended. Calling the `info()` method is possible to see that all dataset share the same columns that are:
- DATA: date of the contract
- GENERE: sex of teh person
- ETA: age of the person
- SETTOREECONOMICODETTAGLIO: category of work
- TITOLOSTUDIO: level of education of the person
- CONTRATTO: type of contract
- MODALITALAVORO: work mode
- PROVINCIAIMPRESA: province of the place of work?
- ITALIANO: nationality of the person

From this first look at the data is possible to see that the dataset `Rapporti_di_lavoro_attivati` is bigger than the other one and this can itroduce some sort of bias in specific type of analysis

### Changing data type
Almost all columns of the dataset are type object, in the following part we will convert every column in an appropriate type.

Starting from the column `DATA` that has been converted whilie importing the data from the csv a part for `Rapporti_di_lavoro_cessati.csv` that has not been converted and so it will be converted now.
Then all the other column except for `ETA` will be converted to strings

In [ ]:
rap_lavoro_attivati[['GENERE','SETTOREECONOMICODETTAGLIO','TITOLOSTUDIO','CONTRATTO', 'MODALITALAVORO','PROVINCIAIMPRESA','ITALIANO']]=rap_lavoro_attivati[['GENERE','SETTOREECONOMICODETTAGLIO','TITOLOSTUDIO','CONTRATTO', 'MODALITALAVORO','PROVINCIAIMPRESA','ITALIANO']].astype('string')

as is possible to see now all the column have a specific data type

In [ ]:
rap_lavoro_attivati.info()

### Searching wrong and null data
Now we will look inside the data checking null or possibly wrong value, after those data will be corrected or deleted.
First off we will find out how many null value there are in each column

In [ ]:
print([rap_lavoro_attivati.isnull().sum()])

From this first look is possible to see that a big part of value is missing from the column `MODALITALAVORO` of the dataset `Rapporti_di_lavoro_attivati` these value needs to be replaced while the other could be simply deleted because they represent a small part of the dataset.

But before proceding now will be checked the actual value of some columnn to see if there are some non plausible data.
Starting from the column `DATA`

In [ ]:
rap_lavoro_attivati['DATA'].describe(datetime_is_numeric=True)

In [ ]:
rap_lavoro_attivati = rap_lavoro_attivati[rap_lavoro_attivati['DATA'] < np.datetime64('2022-02-20')]

In [ ]:
rap_lavoro_attivati['DATA'].describe(datetime_is_numeric=True)

In [ ]:
rap_lavoro_attivati['DATA'].max

In [ ]:
rap_lavoro_attivati = rap_lavoro_attivati[(rap_lavoro_attivati.DATA > '2017') & (rap_lavoro_attivati.DATA < '2020')]

In [ ]:
rap_lavoro_attivati.head()

In [ ]:
rap_lavoro_attivati.info()

In [ ]:
settore_economico = series_to_set('SETTOREECONOMICODETTAGLIO', rap_lavoro_attivati)
len(settore_economico)

Since the `SETTOREECONOMICODETTAGLIO` column has `1125` different values belonging to it, it cannot be easily exploited to analyze data (e.g. dividing them into categories). Furthermore, there are `2888` null values in this column that can make the analysis of this feature even tougher. Considering this two facts, we can derive that dropping the whole column is more convenient than keeping it, since it introduce noise. 

In the `rap_lavoro_attivati` there are a lot of null values for the column `MODALITALAVORO`, removing the rows would reduce the set of data we are analysing. Hence, we'd rather prefer to fill those null values with a suitable values. 

In [ ]:
mod_lavoro = series_to_set('MODALITALAVORO', rap_lavoro_attivati)
mod_lavoro

In [ ]:
settori_eco = rap_lavoro_attivati['SETTOREECONOMICODETTAGLIO'].fillna('')
sett_occur = dict()
settori_eco = list(settori_eco)
settori = set(settori_eco)
for settore in settori:
    sett_occur[settore] = settori_eco.count(settore)

In [ ]:
common = {k: v for k, v in reversed(sorted(sett_occur.items(), key=lambda item: item[1]))}
common = list(common)[:50]

In [ ]:
common[:5]

In [ ]:
rap_lavoro_attivati = rap_lavoro_attivati[
    rap_lavoro_attivati.SETTOREECONOMICODETTAGLIO.isin(common)]
rap_lavoro_attivati

In [ ]:
bins = np.arange(14, 68, 3).tolist()
bins

In [ ]:
rap_lavoro_attivati['agerange'] = pd.cut(rap_lavoro_attivati['ETA'], bins)
rap_lavoro_attivati

The value `NON DEFINITO` can be used to fill the na, since that won't introduce much bias, differently from the other available values. 

In [ ]:
def clean(df):
    df['MODALITALAVORO'] = df['MODALITALAVORO'].fillna('NON DEFINITO')
    df.dropna(axis = 0, inplace = True)
    

clean(rap_lavoro_attivati)
rap_lavoro_attivati.isnull().sum()

#### Working on column `DATA`
In this section we will analyze in more detail the column `DATA` and adjust the data accordingly.
The first step is to group the column by year and count how many contract are activated per year

In [ ]:
print(rap_lavoro_attivati.groupby(pd.Grouper(key='DATA', freq='Y'))['DATA'].count())

as is possible to see meany of the value have date between 2009 and 2022, so to have a more significant set of data we will remove all the row with date previous to 2009

In [ ]:
rap_lavoro_attivati = rap_lavoro_attivati[(rap_lavoro_attivati["DATA"] >= '2009')]
attivatiTemp = rap_lavoro_attivati[(rap_lavoro_attivati["DATA"] >= '2009')]

In [ ]:
print(attivatiTemp.groupby(pd.Grouper(key='DATA', freq='Y'))['DATA'].count())

With this we can see how many contract have been activated in the last few years in the Lombardy region. But before that we apply the same changes to the other datast and after that we can compare the result

In [ ]:
attivatiTemp.groupby(pd.Grouper(key='DATA', freq='Y'))['DATA'].count().plot(label="attivati")
plt.legend()
plt.show()

is possible to see that *attivati* and *cessati*  have a simila behavior but in a different scale while *trasformati* and *prorogati* differs from the previous but have some similarity between them.

In [ ]:
m_to_f_rationA = rap_lavoro_attivati.GENERE.value_counts()

xaxisA = m_to_f_rationA.index
valueA = m_to_f_rationA.values

fig1, axis = plt.subplots(1)
fig1.dpi = 200

#Attivati male to female Graph
axis.pie(valueA, labels=xaxisA, autopct='%1.1f%%', startangle=90)
axis.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.


from these graphs we can say that probably there are more male than female in the popoulation

## Introducing another dataset
This dataset contain the level of education for age referred to the region lombardy from the year 2018 to 2020.The educational level  are coded in teh following way:
- NED = nessun titolo di studio
- IL =  analfabeti
- LBNA = analfabeti privi di titolo di studio
- PSE = licenza di scuola elementare
- LSE = licenza media o avviamento professionale (conseguito non oltre l'anno 1965)/ Diploma di istruzione secondaria di I grado
- USE_IF = Diploma di istruzione secondaria di II grado o di qualifica professionale (corso di 3-4 anni) compresi IFTS
- BL = Diploma tecnico superiore ITS o titolo di studio terziario di primo livello
- ML_RDD = titolo di studio terziario di secondo livello e dottorato di ricerca
- ML = Titolo di studio terziario di secondo livello
- RDD = Dottorato di ricerca/diploma accademico di formazione alla ricerca
- ALL = totale

In [ ]:
grado_istruzione_age  = pd.read_csv(PurePath('dataset', 'Grado_istruzione_per_età_Lombardia_IT1,DF_DCSS_ISTR_LAV_PEN_2_REG.csv'),low_memory=False)
grado_istruzione_age.head()

there are some column with null value that can be deleted

In [ ]:
grado_istruzione_age = grado_istruzione_age[['REF_AREA', 'GENDER', 'AGE_NOCLASS', 'EDU_ATTAIN', 'TIME_PERIOD', 'OBS_VALUE']]

In [ ]:
grado_istruzione_age ['AGE_NOCLASS'].value_counts()

those value represet age range in specific:
- Y_GE9: all people with age greater than 9, in this case all the people in the dataset
- Y25-49: people with an age between 25 and 49
- Y50-64: people with an age between 50 and 64
- Y_GE65: people with more than 65 years
- Y9-24: people with an age between 9 and 24

In the following part we will add a column with the saem age range of the dataset *Grado_istruzione_per_età_Lombardia*

In [ ]:
bins = [0,25, 50, 65, 200]
labels = ['Y9-24', 'Y25-49', 'Y50-64','Y_GE65' ]
rap_lavoro_attivati['agerange'] = pd.cut(rap_lavoro_attivati['ETA'], bins, labels = labels,include_lowest = True)
rap_lavoro_attivati

because in the dataset *rapporti di lavoro* the minimum age is 16 while in *Grado_istruzione_per_età_Lombardia* is 9 we will delete all the illiterate and the people with only elementary school license with the age between 9 and 24  from the dataset, we have choose to do so because the majority of people in that age range with only elementary school license are the one under 16 year

In [ ]:
grado_istruzione_age = grado_istruzione_age [~((grado_istruzione_age ['EDU_ATTAIN']== 'IL')|(grado_istruzione_age ['EDU_ATTAIN']== 'LBNA')|((grado_istruzione_age ['EDU_ATTAIN']== 'PSE')&(grado_istruzione_age ['AGE_NOCLASS']== 'Y9_24')))]
grado_istruzione_age

In [ ]:
print(rap_lavoro_attivati['TITOLOSTUDIO'].head())
grado_istruzione_age['EDU_ATTAIN'].head()

The series `TITOLO_STUDIO` and `EDU_ATTAIN` don't match easily, but we can map the values of `EDU_ATTAIN` into `TITOLO_STUDIO`.

In [ ]:
print(series_to_set('TITOLOSTUDIO', rap_lavoro_attivati))
print(series_to_set('EDU_ATTAIN', grado_istruzione_age))

In [ ]:
grado_istruzione_age.loc[(grado_istruzione_age.EDU_ATTAIN.isin(['ML_RDD'])) & ~(grado_istruzione_age.AGE_NOCLASS == 'Y_GE9')]

In [ ]:
grado_istruzione_age = grado_istruzione_age[grado_istruzione_age['TIME_PERIOD'] != 2020]
grado_istruzione_age = grado_istruzione_age[~grado_istruzione_age.EDU_ATTAIN.isin(['ALL', 'ML', 'RDD'])]
grado_istruzione_age = grado_istruzione_age[grado_istruzione_age.AGE_NOCLASS != 'Y_GE9']
grado_istruzione_age = grado_istruzione_age[grado_istruzione_age.GENDER != 'T']
grado_istruzione_age.head()

In [ ]:
edu_map = {
    'IL': 'NESSUN TITOLO DI STUDIO',
    'NED': 'NESSUN TITOLO DI STUDIO',
    'LBNA': 'NESSUN TITOLO DI STUDIO',
    'PSE': 'LICENZA ELEMENTARE',
    'LSE': 'LICENZA MEDIA',
    'USE_IF': 'DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE',
    "TITOLO DI ISTRUZIONE SECONDARIA SUPERIORE (SCOLASTICA ED EXTRA-SCOLASTICA) CHE NON PERMETTE L'ACCESSO ALL'UNIVERSITÀ ()": 'DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE',
    "DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE  CHE PERMETTE L'ACCESSO ALL'UNIVERSITA": 'DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE',
    'BL': 'LAUREA',
    'ML_RDD': 'TITOLO DI STUDIO TERZIARIO DI SECONDO LIVELLO O DOTTORATO',
    'DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO': 'TITOLO DI STUDIO TERZIARIO DI SECONDO LIVELLO O DOTTORATO',
    'MASTER UNIVERSITARIO DI PRIMO LIVELLO': 'TITOLO DI STUDIO TERZIARIO DI SECONDO LIVELLO O DOTTORATO',
    'LAUREA - Vecchio o nuovo ordinamento': 'LAUREA',
    'DIPLOMA DI SPECIALIZZAZIONE': 'TITOLO DI STUDIO TERZIARIO DI SECONDO LIVELLO O DOTTORATO',
    'DIPLOMA UNIVERSITARIO': 'LAUREA'   
}


In [ ]:
def mapping(series): 
    series = series.apply(lambda x: edu_map.get(x) if edu_map.get(x) != None else x)
    return series

rap_lavoro_attivati['TITOLOSTUDIO'] = mapping(rap_lavoro_attivati['TITOLOSTUDIO'])

grado_istruzione_age['EDU_ATTAIN'] = mapping(grado_istruzione_age['EDU_ATTAIN'])

grado_istruzione_age['EDU_ATTAIN']


In [ ]:
rap_lavoro_attivati['TITOLOSTUDIO']

In the next section we'll see a graph comparing the number of contract activated and the tota number of population in 2018

In [ ]:
import datetime as dt

activate2018 = rap_lavoro_attivati[rap_lavoro_attivati['DATA'].dt.year == 2018]
istr2018 = grado_istruzione_age[(grado_istruzione_age['TIME_PERIOD']== 2018)]
total = istr2018.groupby(['AGE_NOCLASS']).sum()


testAge = activate2018['agerange'].value_counts()

labels = testAge.index
newContract = np.log(testAge.values)
totalPopulation = np.log(total['OBS_VALUE'].values)

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, newContract, width, label='New Contract')
rects2 = ax.bar(x + width/2, totalPopulation, width, label='Total Population')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('log number')
ax.set_title('number by attivati vs total number')
ax.set_xticks(x)
ax.set_xticklabels(labels=labels,rotation=45,
    horizontalalignment='right');
ax.legend()



fig.tight_layout()

plt.show()

In [ ]:
city_codes = pd.read_excel(PurePath('dataset', 'Elenco-comuni-italiani.xls'))
city_codes = city_codes[['Codice Comune formato alfanumerico', 
                        'Denominazione in italiano',
                        'Denominazione Regione',
                        'Provincia',
                        'Codice NUTS3 2021',
                        'Codice NUTS2 2021 (3) '
                        ]]
city_codes = city_codes[city_codes['Denominazione Regione'] == 'Lombardia']
city_codes = city_codes.set_index('Codice Comune formato alfanumerico')
city_codes.loc[1] = ['','','Milano','ITC45','']   #The code in the dataset does not match with the code of Milano, 
                                                  #manually added
city_codes.head()

In [ ]:
def get_provincia_by_code(code):
    return city_codes[city_codes['Codice NUTS3 2021'] == code]['Provincia'].iloc[0]
                    

In [ ]:
grado_istruzione_age = grado_istruzione_age[grado_istruzione_age.REF_AREA.isin(city_codes['Codice NUTS3 2021'])]

In [ ]:
grado_istruzione_age['REF_AREA'] = grado_istruzione_age['REF_AREA'].apply(lambda x: get_provincia_by_code(x)) 
grado_istruzione_age

In [ ]:
grado_istruzione_age[grado_istruzione_age.REF_AREA == 'Milano'].groupby(['REF_AREA', 'TIME_PERIOD', 'GENDER', 'EDU_ATTAIN', 'AGE_NOCLASS']).sum().head(60)

In [ ]:
series_to_set('REF_AREA', grado_istruzione_age)